In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM,Dropout,Bidirectional
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
PATH ='C:/Users/ckdrj/Google 드라이브/baseball/data/total/'
#PATH = 'C:/Users/LIM CHANG GEON/Google 드라이브/baseball/data/total'
hiter16 = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2016.csv")[["GDAY_DS","T_ID","HIT","AB"]]  #"T_ID","HEADER_NO"
pitcher16 = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2016.csv")[["GDAY_DS","T_ID","ER","INN2","WLS"]]

In [3]:
hiter17 = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2017.csv")[["GDAY_DS","T_ID","HIT","AB"]]
pitcher17 = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2017.csv")[["GDAY_DS","T_ID","ER","INN2","WLS"]]

In [4]:
hiter18 = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2018.csv")[["GDAY_DS","T_ID","HIT","AB"]]
pitcher18 = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2018.csv")[["GDAY_DS","T_ID","ER","INN2","WLS"]]

In [5]:
hiter19 = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2019.csv")[["GDAY_DS","T_ID","HIT","AB"]]
pitcher19 = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2019.csv")[["GDAY_DS","T_ID","ER","INN2","WLS"]]

In [6]:
hiter20 = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2020.csv")[["GDAY_DS","T_ID","HIT","AB"]]
pitcher20 = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2020.csv")[["GDAY_DS","T_ID","ER","INN2","WLS"]]

In [7]:
hiter = pd.concat([hiter16,hiter17,hiter18,hiter19,hiter20])
del hiter16,hiter17,hiter18,hiter19,hiter20
hiter['AVG']= hiter['HIT']/hiter['AB']
hiter["y_m"]=hiter.GDAY_DS.astype(str).str[2:6]
del hiter['HIT'], hiter['AB'], hiter["GDAY_DS"]
hiter.head()

,T_ID,AVG,y_m
0,LG,0.214286,1604
1,HH,0.282609,1604
2,NC,0.300000,1604
3,HT,0.235294,1604
4,SK,0.277778,1604


In [8]:
pitcher = pd.concat([pitcher16,pitcher17,pitcher18,pitcher19,pitcher20])
del pitcher16,pitcher17,pitcher18,pitcher19,pitcher20
pitcher["ERA"] = 27*pitcher.ER/pitcher.INN2
pitcher.WLS = pitcher.WLS.map({'W':1,'D':0.5,"L":0})
pitcher["y_m"]=pitcher.GDAY_DS.astype(str).str[2:6]
del pitcher["ER"], pitcher["INN2"],pitcher["GDAY_DS"]
pitcher.head()

,T_ID,WLS,ERA,y_m
0,LG,1.0,3.000000,1604
1,HH,0.0,3.176471,1604
2,NC,1.0,4.000000,1604
3,HT,0.0,5.625000,1604
4,SK,0.0,8.000000,1604


In [9]:
team_list = ['WO','OB','NC','SK','LG','KT','LT','HT','SS','HH']
df_mean=pd.DataFrame()
code=0
for team in team_list:
    code +=0.1
    pitcher_team= pitcher[pitcher.T_ID==team]
    hiter_team=hiter[hiter.T_ID==team]
    pitcher_mean = pitcher_team.groupby(['y_m']).mean()
    pitcher_mean["count"] =pitcher_team.groupby(['y_m']).count().T_ID
    hiter_mean = hiter_team.groupby(['y_m']).mean()
    df = pd.concat([pitcher_mean, hiter_mean], axis=1)
    df["team_code"] = code
    print(df.shape)
    df_mean=pd.concat([df_mean,df])

(34, 5)
(34, 5)
(34, 5)
(33, 5)
(33, 5)
(33, 5)
(34, 5)
(33, 5)
(33, 5)
(33, 5)


In [10]:
df_mean.reset_index(inplace=True, drop=True)

In [11]:
df_mean = df_mean[df_mean["count"] > 5]
df_mean.reset_index(inplace=True, drop=True)

In [12]:
del df_mean["count"]

In [13]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
sc_era = MinMaxScaler()
sc_avg = MinMaxScaler()
df_mean["AVG"]=sc_avg.fit_transform(df_mean[["AVG"]])
df_mean["ERA"]=sc_era.fit_transform(df_mean[["ERA"]])

In [14]:
df_mean.head()

,WLS,ERA,AVG,team_code
0,0.479167,0.307739,0.410674,0.1
1,0.560000,0.397596,0.409027,0.1
2,0.560000,0.584479,0.562168,0.1
3,0.666667,0.467362,0.634089,0.1
4,0.565217,0.390089,0.518983,0.1


In [15]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

In [16]:
model=Sequential()
model.add(Dense(20,activation='relu',input_dim=3))
model.add(Dense(5,activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

W0920 01:59:02.221695  1868 deprecation_wrapper.py:119] From F:\sda\envs\dlstudy\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0920 01:59:02.273658  1868 deprecation_wrapper.py:119] From F:\sda\envs\dlstudy\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0920 01:59:02.277660  1868 deprecation_wrapper.py:119] From F:\sda\envs\dlstudy\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0920 01:59:02.313654  1868 deprecation_wrapper.py:119] From F:\sda\envs\dlstudy\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [17]:
df_mean=df_mean.sample(frac=1).reset_index(drop=True)

In [18]:
x_train=df_mean.iloc[:240,1:]
y_train=df_mean.iloc[:240,0]
x_test=df_mean.iloc[240:,1:]
y_test=df_mean.iloc[240:,0]

In [19]:
x_train.shape

(240, 3)

In [20]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
model.reset_states()
model.fit(x_train, y_train, epochs=100, validation_split=0.15,callbacks=[early_stop])

W0920 01:59:02.905258  1868 deprecation_wrapper.py:119] From F:\sda\envs\dlstudy\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0920 01:59:02.979238  1868 deprecation_wrapper.py:119] From F:\sda\envs\dlstudy\lib\site-packages\keras\backend\tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Train on 204 samples, validate on 36 samples
Epoch 1/100
204/204 [==============================] - 0s 912us/step - loss: 0.0920 - val_loss: 0.0956
Epoch 2/100
204/204 [==============================] - 0s 39us/step - loss: 0.0699 - val_loss: 0.0716
Epoch 3/100
204/204 [==============================] - 0s 29us/step - loss: 0.0531 - val_loss: 0.0543
Epoch 4/100
204/204 [==============================] - 0s 39us/step - loss: 0.0447 - val_loss: 0.0440
Epoch 5/100
204/204 [==============================] - 0s 34us/step - loss: 0.0408 - val_loss: 0.0382
Epoch 6/100
204/204 [==============================] - 0s 39us/step - loss: 0.0379 - val_loss: 0.0336
Epoch 7/100
204/204 [==============================] - 0s 34us/step - loss: 0.0342 - val_loss: 0.0299
Epoch 8/100
204/204 [==============================] - 0s 34us/step - loss: 0.0304 - val_loss: 0.0269
Epoch 9/100
204/204 [==============================] - 0s 29us/step - loss: 0.0271 - val_loss: 0.0244
Epoch 10/100
204/204 [==============

In [21]:
pre = model.predict(x_test)
model.evaluate(x_test,y_test)

58/58 [==============================] - 0s 35us/step


0.007353849355773679

In [22]:
ny_test=y_test.values.reshape(y_test.shape[0],1)
np.concatenate((pre,ny_test),axis=1)

array([[0.4365027 , 0.5       ],
       [0.41713598, 0.45454545],
       [0.53068691, 0.52173913],
       [0.37801242, 0.43478261],
       [0.49207187, 0.38095238],
       [0.55358505, 0.71428571],
       [0.46288097, 0.43478261],
       [0.5230521 , 0.56818182],
       [0.60731983, 0.72      ],
       [0.46917063, 0.29166667],
       [0.35508314, 0.41666667],
       [0.33926272, 0.42      ],
       [0.35152778, 0.29166667],
       [0.52160627, 0.5       ],
       [0.35257405, 0.39130435],
       [0.48140407, 0.65384615],
       [0.62575215, 0.625     ],
       [0.45426261, 0.5       ],
       [0.54428655, 0.5       ],
       [0.48901612, 0.58333333],
       [0.55503702, 0.65384615],
       [0.6527065 , 0.59090909],
       [0.45872611, 0.28571429],
       [0.52050364, 0.52083333],
       [0.36562166, 0.33333333],
       [0.4782362 , 0.54545455],
       [0.58328766, 0.61904762],
       [0.5053336 , 0.53846154],
       [0.61268783, 0.6       ],
       [0.6398629 , 0.7826087 ],
       [0.